In [21]:
import os
from data_preparation import *
from model import *


In [22]:
BATCH_SIZE = 16
BUFFER_SIZE = 10000
dataset, char2idx, idx2char, vocab = word_text2tf_data(filename='donald_tweets.txt',
                                                       seq_length=20,
                                                       batch_size=BATCH_SIZE,
                                                       buffer_size=BUFFER_SIZE)

In [23]:
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNIT = 256
model = build_attention_model(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    rnn_units=RNN_UNIT,
    batch_size=BATCH_SIZE)
model.summary()

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(16, None)]         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (16, None, 256)      5555712     input_5[0][0]                    
                                                                 input_5[0][0]                    
__________________________________________________________________________________________________
gru_4 (GRU)                     (16, None, 256)      394752      embedding_4[0][0]                
                                                                 embedding_4[1][0]                
_______________________________________________________________________________________

In [24]:
LOSS = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
OPTIMIZER = 'adam'
model.compile(optimizer=OPTIMIZER, loss=LOSS)

In [25]:
checkpoint_dir = './attention_training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
tf.train.latest_checkpoint(checkpoint_dir)

Epoch 1/10
385/385 [==============================] - 14s 35ms/step - loss: 7.6981

'./attention_training_checkpoints\\ckpt_10'

In [26]:
model = build_attention_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNIT, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))


In [27]:
def generate_text(model, start_string, num_generate, split_string):
    start_string
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)


    text_generated = []
    temperature = 1.0

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    return split_string.join(start_string) + split_string.join(text_generated)


print(generate_text(model, ['@ABCPolitics:'], 1000, ' '))

@ABCPolitics:thought that media would not be able to discuss! 
 "@smithsj1: @realDonaldTrump you swear for rising and how they did so big numbers - children. loves them to the students of @TIME story. Totally only thats the won! 
 "@JoeNBC: Latest Poll just came out Trump down, lead in the primary campaign higher than Bernie Sanders "political gunman, - There are a good man. I beat Hillary Clinton campaign Thursday: primary campaign is DELUDED! is gotten to happen under Crooked Crooked Hillarys email Foundation- president." 
 Respectfully, you than the way that Hillary Clinton is off a OPSEC America." 
 The worst Partnership and John Kasich native chairman of the State of the polls like Crooked Hillary to report the front of my book CRIPPLED AMERICA. Don۪t look forward forward to, 
 "@reganharycki: @realDonaldTrump in OAN and left change 
 Join me live from Columbus, Ohio problem for you live @ April California- very special! They will fight questions about all! 
 Thank you notice the 

In [29]:
from bleu import *
bleu_score(2,10,generate_text,model)

['RT', '@DonaldJTrumpJr:']
0.12978984338769176
